In [3]:
import numpy as np

In [4]:
def xsquared(x):
    return x**2

In [5]:
def trapezoid(func,lbound,ubound,N=100):
    """
    Calculate the integral of a function using the trapezoid rule
    with equal spaced abscissae
    
    func -- function which gives the y values
    lbound -- lower bound of integral
    ubound -- upper bound of integral
    N      -- number of abscissae
    
    """
    xvalues = np.linspace(lbound,ubound,N)
    delta_x = (ubound-lbound)/(N-1)
    # first point
    integral = func(lbound)/2
    # all middle points
    for x in xvalues[1:-1]:
        integral += func(x)
    # endpoint
    integral += func(xvalues[-1])/2
    integral *= delta_x
    
    return integral
    
    
    
    

In [6]:
def simpsons(func, lbound, ubound, N=100):
    """
    Calculate the integral of a function using Simpsons rule
    with equal spaced abscissae
    
    func -- function which gives the y values
    lbound -- lower bound of integral
    ubound -- upper bound of integral
    N      -- number of abscissae (N-1 intervals)
    
    """
    xvalues = np.linspace(lbound,ubound,N)
    delta_x = (ubound-lbound)/(N-1)
    # first point and last point 
    integral = func(lbound) + func(ubound)
    # all other points
    for i in range(1,N-1):
        # om en om 4 en 2, beginnen bij 4 als i=1
        if i % 2 == 0:            
            integral += 2*func(xvalues[i])
        else:
            integral += 4*func(xvalues[i])
    integral *= delta_x/3
    
    return integral
    

In [16]:
def open_simpsons(func, lbound, ubound, N=100, lb_closed=False, ub_closed=False):
    # if both bounds are closed, just do normal simpsons
    if lb_closed and ub_closed:
        return simpsons(func,lbound,ubound,N)
    
    delta_x = (ubound-lbound)/(N-1)
    xvalues = np.linspace(lbound,ubound,N)
    
    if not lb_closed: # If lb open, first point is now x1 
        xvalues = xvalues[1:]
        
    if not ub_closed: # last point is N-1
        xvalues = xvalues[:-1]
        
    # coefficients
    if lb_closed: # if lb closed use 'normal' coefficients
        coefficients = [1,4,2,4] #x0 till x3
    else:
        coefficients = [27/4,0,13/4] #x1 till x3 
    for i in range(4,N-4): # go until N-3
        # 4 and 2 alternating
        if i % 2 == 0:
            coefficients.append(4)
        else:
            coefficients.append(2)
    
    if ub_closed: # if ub closed use 'normal' coefficients
        coefficients += [4,2,4,1] # N-3 to N
    else:
        coefficients += [13/4,0,27/4] # N-3 to N-1
        
    coefficients = np.asarray(coefficients)
    
    integral = delta_x/3 * np.sum(func(xvalues)*coefficients)
    
    return integral
    

In [17]:
def romberg(func, lbound, ubound, order=6):
    """
    Calculate the integral of a function using Romberg's method
    with equal spaced abscissae
    
    func -- function which gives the y values
    lbound -- lower bound of integral
    ubound -- upper bound of integral
    N      -- number of abscissae
    
    """
    
    
    # for saving S_i,j's
    all_S = np.zeros((order,order))
    
    i = 0
    delta_x = (ubound-lbound)
    points = np.linspace(lbound,ubound,2**i+1)
    integral = delta_x/2 * np.sum(func(points))
    all_S[0,0] = integral
    
    # Then calculate the first column (S_{i,0})
    for i in range(1,order):
        delta_x /= 2
        # add points in the middle
        points = np.linspace(lbound,ubound,2**i+1)
        # add new points to the integral (om en om, starting from 1)
        integral = 0.5*integral + delta_x * np.sum(func(points[1::2]))
        
        all_S[i,0] = integral
    
    for j in range(1,order): # columns
        for i in range(j,order): # rows
            #print (i,j)
            #print (f'{4**j}*S{i},{j-1} - S{i-1},{j-1} / {4**j} - 1' )
            all_S[i,j] = (4**j*all_S[i,j-1] - all_S[i-1,j-1]) / (
                           4**j -1)

    # compare this for the error function with the slides. 
    # print (all_S)
    
    return all_S[order-1,order-1]
        

    
    
    

In [21]:
q1a = trapezoid(xsquared,1,5,N=100)
q1a2 = trapezoid(lambda x: np.sin(x), 0, np.pi, N=100)

print ('\n')
print ("Trapezoid")
print ("Question 1a, x**2: ", q1a)
print ("Question 1a, sin(x)", q1a2)

q1a = simpsons(xsquared,1,5,N=100)
q1a2 = simpsons(lambda x: np.sin(x), 0, np.pi, N=100)

print ('\n')
print ("Simpsons")
print ("Question 1a, x**2: ", q1a)
print ("Question 1a, sin(x)", q1a2)

q1a = romberg(xsquared,1,5,order=6)
q1a2 = romberg(lambda x: np.sin(x), 0, np.pi, order=6)

print ('\n')
print ("Romberg")
print ("Question 1a, x**2: ", q1a)
print ("Question 1a, sin(x)", q1a2)

test = romberg(lambda x: 2/np.sqrt(np.pi) * np.exp(-x**2),0,1,order=5)
print ('Error function', test)




Trapezoid
Question 1a, x**2:  41.334421657654005
Question 1a, sin(x) 1.9998321638939924


Simpsons
Question 1a, x**2:  40.99935380743462
Question 1a, sin(x) 1.9998321638939922


Romberg
Question 1a, x**2:  41.333333333333336
Question 1a, sin(x) 2.000000000001321
Error function 0.8427007932686706
